# Day 1 Exercises

## Table of Contents
<a name="toc"></a>
1. [$f(x) = Ax^n$](#Function)
2. [$f(x) = \exp(x)$](#Exp) 
3. [Harmonic Potential Force.](#force) 
4. [Euler integrator](#Euler) 
5. [Verlet/Leapfrog integrator](#Verlet)
6. [Runge-Kutta integrator](#RK4)
7. [Simpson's rule](#Simpson's)

In [1]:
import h5py
import numpy as np
from math import factorial

## 1. Function that computes $f(x) = Ax^n.$ <a name="Function"></a>
[Return to Table of contents](#toc)

In [5]:
def Ax_n(parms):
    """
    Inputs:
    1. parms - a dictionary containing the coefficient, A, the value of x, and the integer exponent n
    
    Outputs:
    1. returns the value of Ax^n
    """
    
    A = parms["A"]
    x = parms["x"]
    n = parms["n"]
    
    if (n == 0):
        return A
    
    elif (n > 0):
        total = 1.0
        ctr = 0
        while (ctr < n):
            total = total * x
            ctr = ctr + 1
            
        return A * total
    
    else:
        total = 1.0
        ctr = 0
        while (ctr < np.abs(n)):
            total = total / x
            ctr = ctr + 1
            
        return A * total

In [6]:
# test
parms = { "A":5.0, "x":2.0, "n":1 }
print(Ax_n(parms))

10.0


## 2. Function that computes $\exp(x) \approx 1 + x + \frac{x^2}{2!} + \cdots + \frac{x^n}{n!}$ <a name="Exp"></a>
[Return to Table of contents](#toc)

In [7]:
def exp(params):
    """
    Inputs:
    1. params - a dictionary containing the specific value which we evaluate e^x at as well as the order to which
                we would like to be accurate, n
    
    Outputs:
    1. returns the value of e^x
    """
    # first make sure that the order is sensible
    n = params["order"]
    if (n < 0):
        return "Invalid entry, please enter nonnegative orders."
    
    # compute e^x
    else:
        x = params["x"]
        total = 0.0
        
        # the part that actually does the computation, using the function above, Ax^n
        for i in range(n):
            temp = Ax_n({ "A":1.0, "x":x, "n":i }) / factorial(i)
            total = total + temp
            
        return total

In [8]:
# test
params = {"x":3.0, "order":4}
print(exp(params))

13.0


## 3. Harmonic potential <a name="force"></a>
[Return to Table of contents](#toc)

We will assume the force is derived from a harmonic potential in unless stated otherwise:


\begin{align*}
    V(\mathbf{q}) = \dfrac{1}{2}\sum_j k_j(q_j - q_{j,0})^2.
\end{align*}

With this in mind, 

\begin{align*}
    F_j(t) = -\dfrac{\partial V}{\partial q_j} = - k_j (q_j - q_{j,0}).
\end{align*}

In [9]:
def get_force(q, params):
    """
    Returns the force derived from a harmonic potential.
    
    Inputs:
    1. q      - coordinates of the harmonic oscillator
    2. params - dictionary with mass, equilibrium position, and force constant of the harmonic oscillator
    
    Outputs:
    1. returns force
    """
    
    force_const = params["force_const"]
    equi_pos = params["q0"]
    
    return - force_const * (q - equi_pos)

## 4. Euler integration <a name="Euler"></a>
[Return to Table of contents](#toc)

This integrator updates the positions and momenta of the selected degrees of freedom using the Euler Integration scheme:

\begin{align*}
    q_k(t + \Delta t) &= q_k(t) + \dfrac{\Delta t}{m_k} \cdot p_k(t)\\
    p_k(t + \Delta t) &= p_k(t) + \Delta t \cdot F_k(t).
\end{align*}

In [10]:
# Used the RK4 tutorial as a guide to set up efficiently
def euler_update(q, p, params, dt):
    """
    Updates the positions and momenta of the harmonic oscillator using a for loop.
    Inputs:
    1. q      - coordinates of the harmonic oscillator
    2. p      - momenta of the harmonic oscillator
    3. params - dictionary with mass, equilibrium position, and force constant of the harmonic oscillator
    4. dt     - time-step 
    
    Outputs:
    1. returns the updated positions and momenta 
    """
    force_const = params["force_const"]
    equi_pos = params["q0"]
    mass = params["mass"]
    ndof = len(q)
    
    for dof in range(ndof):
        force = - force_const * (q[dof] - equi_pos)
        q[dof] = q[dof] + (dt * p[dof] / mass)
        p[dof] = p[dof] + (dt * force)
        
    return q, p

In [11]:
# test
a = np.array([1.0, 1.0])
b = np.array([1.0, 1.0])
euler_params = {"force_const":1.0, "q0": 0.0, "mass":1.0} # dictionary
dt = 1.0
print(euler_update(a, b, euler_params, dt))

(array([2., 2.]), array([0., 0.]))


In [12]:
def euler_update_vectorized(q, p, params, dt):
    """
    Updates the positions and momenta of the harmonic oscillator without direct loops.
    Inputs:
    1. q      - coordinates of the harmonic oscillator
    2. p      - momenta of the harmonic oscillator
    3. params - dictionary with mass, equilibrium position, and force constant of the harmonic oscillator
    4. dt     - time-step 
    
    Outputs:
    1. returns the updated positions and momenta 
    """
    
    mass = params["mass"]
    
    force = get_force(q, params)
    
    q = q + (dt * p) / mass
    p = p + (dt * force)
    
    return q, p

In [13]:
# test
a = np.array([1.0, 1.0])
b = np.array([1.0, 1.0])
euler_params = {"force_const":1.0, "q0": 0.0, "mass":1.0}
dt = 1.0
print(euler_update_vectorized(a, b, euler_params, dt))

(array([2., 2.]), array([0., 0.]))


## 5. Verlet/Leapfrog Integration <a name="Verlet"></a>
[Return to Table of contents](#toc)

This integrator updates the positions and momenta of the selected degrees of freedom using the Verlet Integration scheme:

\begin{align*}
    q_k(t + \Delta t) &= 2q(t) - q(t - \Delta t) + \dfrac{2\Delta t^2}{m}F(t)\\
    p_k(t) &= \dfrac{m}{2\Delta t} [q(t + \Delta t) - q(t - \Delta t)].
\end{align*}

In [14]:
def verlet_update(q_current, q_previous, params, dt):
    """
    Updates the positions and momenta of the harmonic oscillator without direct loops.
    
    q(t + dt) = q(t) - q(t - dt) + (F(t) * dt^2)/m
    p(t) = (q(t+dt) - q(t-dt))/(2mdt)
    
    Inputs:
    1. q_current   - current coordinates of the harmonic oscillator, q(t)
    2. q_previous  - previous coordinates of the harmonic oscillator, q(t - dt)
    3. params      - dictionary with mass, equilibrium position, and force constant of the harmonic oscillator
    4. dt          - time-step 
    
    Outputs:
    1. q_update  - the updated positions, q(t + dt)
    2. q_current - the current positions q(t) in this pass will be the "previous" coordinates in the next pass
    3. p         - the momenta p(t) (for tracking)
    """
    
    mass = params["mass"]
    ndof = len(q)
    
    # compute the force
    force = get_force(q, params)
    force = - force_const * (q - equi_pos)
    
    q_update = 2 * q_current - q_previous + (2 * (dt**2) * force) / mass
    p = m * (q_update - q_previous) / (2 * dt)
        
    return q_update, q_current, p

## 6. Runge-Kutta Algorithm <a name="RK4"></a>
[Return to Table of contents](#toc)

We are given a first order differential equation of the form $$ \dfrac{dx}{dt} = f(x,t), $$ subject to $x(t_0) = x_0$ on the interval $[a,b]$. We split the interval into $N$ steps, and for each step ($n\geq0$), but the difference lies in the computation of $x_{n+1}$. Now $x_{n+1} = x_n + h*k$, where $h = (b-a)/N$ and $k = (k_1 + 2k_2 + 2k_3 + k_4)/6$ with

\begin{align*}
    k_1 &= f(t_n,x_n)\\
    k_2 &= f(t_n + 0.5h, x_n + 0.5hk_1)\\
    k_3 &= f(t_n + 0.5h, x_n + 0.5hk_2)\\
    k_4 &= f(t_{n+1}, x_n + hk_3).
\end{align*}

In [2]:
# I wrote this when I first started learning Python -- I will adapt it to this code soon.
# creates an array with 1000 steps from 0 to 10
rktpoints = arange(a,b,h) 

# calculate x_{n+1} and add to the x array
rkxpoints = []
for t in rktpoints:
    rkxpoints.append(x)
    k1 = f(x,t)
    k2 = f(x + 0.5*h*k1, t + 0.5*h)
    k3 = f(x + 0.5*h*k2, t + 0.5*h)
    k4 = f(x + h*k3, t + h)
    k = (k1 + k2 + k3 + k4)/6
    x += h*k
    
# plot the data
plot(rktpoints,rkxpoints,"g--")
xlabel("t")
ylabel("x(t)")
show()

NameError: name 'np' is not defined

## 7. Simpson's rule <a name="Simpson's"></a>
[Return to Table of contents](#toc)
Let $f:[a,b]\rightarrow \mathbb{R}$ be a continuous function. We can divide the interval $[a,b]$ into $n$ partions in the following manner 

\begin{align*}
    a = x_0 < x_1 < x_2 < \cdots < x_{n-1} < x_n = b.
\end{align*}

We define $\Delta x = (b-a)/n$ and construct the sequence $\{S_n \}$ defined by

\begin{align*}
    S_n = \dfrac{\Delta x}{3} \left[ f(x_0) + 4 f(x_1) + 2f(x_2) + 4 f(x_3) + \cdots +2 f(x_{n-2}) + 4 f(x_{n-1}) + f(x_n) \right].
\end{align*}

Simpson's rule ultimately shows 

\begin{align*}
    \lim_{n \rightarrow \infty} S_n = \int_a^b f(x)\, dx.
\end{align*}


<p id="citeText">Numerical Integration - Midpoint, Trapezoid, Simpson’s Rule. 20 Dec. 2020, https://math.libretexts.org/@go/page/10269.
</p>

In [15]:
# assume function has been previously defined
def f(x):
    return -x**2 + x + 3

def integrate_simpsons(params):
    """
    Computes the value of an integral using Simpon's rule, which is outlined above (note the reference at the bottom).
            
    Inputs:
    1. params - a dictionary containing 
                a) a lower bound, lb
                b) an upper bound, ub
                c) the number of desired subintervals
                *note that the function can be defined in the function above.
    
    Outputs:
    1. The numerical value of the integral over the given set. 
    """
    
    b = params["ub"]
    a = params["lb"]
    n = params["subintervals"]
    dx = (b - a) / n
    
    # intialize total to keep track
    total = 0.0
    for i in range(n+1):
        arg = a + (i * dx)
        #array[i] = f(arg)
        
        # in the defining sequence, one is the coefficient of both x_0 and x_n
        if ((i == 0) or (i == n)):
            # replace array[i] with f(arg)
            total = total + f(arg)
            
        # in the defining sequence, the even indexed elements have a coefficient of 2
        elif (i%2 == 0):
            total = total + 2 * f(arg)
        
        # in the defining sequence, the odd indexed elements have a coefficient of 4
        else:
            total = total + 4 * f(arg)
    
    # after summing up, we multiply by dx and divide by 3
    return total * dx / 3
    
# test
simpsons_params = {"lb":0, "ub":2, "subintervals": 1000}
print(integrate_simpsons(simpsons_params))


5.333333333333339
